In [70]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

In [71]:
def get_files(folder):
    filenames = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".png") and "imgs" in root:
                filenames.append(os.path.join(root, file))
    return filenames

In [72]:
files = get_files("data/")
train_paths = [(path, path.replace("imgs", "targets")) for path in files if "train" in path]
test_paths = [(path, path.replace("imgs", "targets")) for path in files if "test" in path]
val_paths = [(path, path.replace("imgs", "targets")) for path in files if "valid" in path]

print(train_paths[0])

print("Number of training samples: ", len(train_paths), 
      "\nNumber of validation samples: ", len(val_paths),
      "\nNumber of test samples: ", len(test_paths))

('data/train/imgs/HIP_0318.png', 'data/train/targets/HIP_0318.png')
Number of training samples:  320 
Number of validation samples:  91 
Number of test samples:  47


In [73]:
def normalize_image(image):
    image = tf.cast(image, tf.float32) - tf.reduce_mean(image)
    image = image / tf.math.reduce_std(image)
    return image

def normalize_mask(mask):
    mask = tf.cast(mask, tf.float32)
    mask = mask / 255.0
    return mask

def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, IMSIZE)
    return image


def load_mask(mask_path):
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, IMSIZE)
    return mask

def load_image_pair(image_path):
    image = load_image(image_path[0])
    mask = load_mask(image_path[1])
    image = normalize_image(image)
    mask = normalize_mask(mask)
    return image, mask

import tensorflow.keras.backend as K
@tf.function
def dice(y_true, y_pred):
    """Computes the Dice loss value between `y_true` and `y_pred`.

    Formula:
    ```python
    loss = 1 - (2 * sum(y_true * y_pred)) / (sum(y_true) + sum(y_pred))
    ```

    Args:
        y_true: tensor of true targets.
        y_pred: tensor of predicted targets.

    Returns:
        Dice loss value.
    """
    y_true = K.cast(y_true, y_pred.dtype)

    inputs = K.flatten(y_true)
    targets = K.flatten(y_pred)
    
    intersection = K.sum(inputs * targets)
    dice = tf.divide(
        2.0 * intersection,
        K.sum(y_true) + K.sum(y_pred) + K.epsilon(),
    )

    return 1 - dice

@tf.function
def dice_metric(y_true, y_pred):
    """Computes the Dice loss value between `y_true` and `y_pred`.

    Formula:
    ```python
    loss = 1 - (2 * sum(y_true * y_pred)) / (sum(y_true) + sum(y_pred))
    ```

    Args:
        y_true: tensor of true targets.
        y_pred: tensor of predicted targets.

    Returns:
        Dice loss value.
    """
    y_true = K.cast(y_true, y_pred.dtype)

    inputs = K.flatten(y_true)
    targets = K.flatten(y_pred)

    intersection = K.sum(inputs * targets)
    dice = tf.divide(
        2.0 * intersection,
        K.sum(y_true) + K.sum(y_pred) + K.epsilon(),
    )

    return dice

class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
        self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
        self.augment_inputs = tf.keras.layers.RandomFlip(mode="vertical", seed=seed)
        self.augment_labels = tf.keras.layers.RandomFlip(mode="vertical", seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels

In [74]:
# Create the dataset
BUFFER_SIZE = 100
BATCH_SIZE = 16
IMSIZE = (256, 256)

def create_dataset(paths):
    dataset = tf.data.Dataset.from_tensor_slices(paths)
    dataset = dataset.map(load_image_pair, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    batches = (
        dataset
        .cache()
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE)
        # .map(Augment(), num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    return batches

train_batches = create_dataset(train_paths)
test_batches = create_dataset(test_paths)
val_batches = create_dataset(val_paths)

In [75]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Layer
from tensorflow.keras.models import Model

default_args = {
    "kernel_initializer": "he_normal",
    "padding": "same",
    "activation": "relu"
}

def unet(dropout_rate=0.2, lr=1e-3, init="he_normal", adam_beta1=0.9, adam_beta2=0.999):
    in1 = Input(shape=(*IMSIZE, 1))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=init, padding='same')(in1)
    conv1 = Dropout(dropout_rate)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(pool1)
    conv2 = Dropout(dropout_rate)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init, padding='same')(pool2)
    conv3 = Dropout(dropout_rate)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init, padding='same')(pool3)
    conv4 = Dropout(dropout_rate)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv4)

    up1 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(up1)
    conv5 = Dropout(dropout_rate)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv5)

    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(up2)
    conv6 = Dropout(dropout_rate)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=init, padding='same')(up2)
    conv7 = Dropout(dropout_rate)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=init, padding='same')(conv7)
    segmentation = Conv2D(1, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr, beta_1=adam_beta1, beta_2=adam_beta2),
                  loss = dice,
                  metrics=['accuracy', dice_metric])
    return model

In [76]:
#model = get_second_model(dropout=0.3, lr=4e-4, init="he_normal")
#model.summary()

In [77]:
#history = model.fit(train_batches,
#                    validation_data=val_batches,
#                    epochs=150,
#                    verbose=2)

In [78]:
# for images, masks in test_batches.take(1):
#     predictions = model.predict(images)
#     fig, axs = plt.subplots(3, 3, figsize=(15, 15))
#     for ii, (img, msk, prediction) in enumerate(zip(images, masks, predictions)):
#         axs[ii, 0].imshow(img, cmap='gray')
#         axs[ii, 1].imshow(msk, cmap='gray')
#         axs[ii, 2].imshow(prediction, cmap='gray')
#         axs[ii, 0].axis("off")
#         axs[ii, 1].axis("off")
#         axs[ii, 2].axis("off")
#         axs[ii, 0].set_title("Image")
#         axs[ii, 1].set_title("Mask")
#         axs[ii, 2].set_title("Prediction")
#         if ii == 2:
#             break

# fig, axs = plt.subplots(1, 2)
# axs[0].plot(history.history['loss'])
# axs[0].plot(history.history['val_loss'])
# axs[1].plot(history.history['dice_metric'])
# axs[1].plot(history.history['val_dice_metric'])
# axs[0].set_title('Model loss')
# axs[0].set_ylabel('Loss')
# axs[0].set_xlabel('Epoch')
# axs[0].legend(['Train', 'Val'], loc='upper left')
# axs[1].set_title('Model accuracy')
# axs[1].set_ylabel('Accuracy')
# axs[1].set_xlabel('Epoch')
# axs[1].legend(['Train', 'Val'], loc='lower right')
# plt.show()

In [79]:
parameters={
    'init': ['he_normal', 'glorot_uniform'],
    'dropout_rate': [0.2, 0.3, 0.5],
    'adam_beta1': [i / 10 for i in range(1,10)],
    'adam_beta2': [0.001 + i / 10 for i in range(1,10)]
}

In [80]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# def create_model(lr=0.001, init='he_normal', dropout_rate= 0.1, adam_beta1=0.9, adam_beta2=0.999):
#     return KerasClassifier(
#         build_fn=get_second_model, 
#         lr=lr,
#         init=init, 
#         dropout_rate=dropout_rate,
#         adam_beta1=adam_beta1,
#         adam_beta2=adam_beta2,
#         epochs=150, 
#         batch_size=32, 
#         verbose=0#
#     )

# random_search = RandomizedSearchCV(estimator=get_second_model(), param_distributions=parameters, n_iter=10, cv=3, verbose=1)
# x_train = np.concatenate([x for x, _ in train_batches], axis=0)
# y_train = np.concatenate([y for _, y in train_batches], axis=0)
# for x, y in train_batches:
#     print(x.shape)
#     print(y.shape)
#     break
# print(x_train.shape)
# print(y_train.shape)

# random_search_result = random_search.fit(x_train, y_train)

In [81]:
determine_learning_rate = False
learning_rate_results = {}

if determine_learning_rate:
    # determine learning rate
    for learning_rate in [10 ** -i for i in range(3,6)]:
        model = unet()
        history = model.fit(train_batches,
                        validation_data=val_batches,
                        epochs=50,
                        verbose=2)
        learning_rate_results[learning_rate] = max(history.history['val_dice_metric'])


In [82]:
learning_rate_results

{}

In [83]:
learning_rate = 0.001

In [84]:
for param in parameters.items():
    print(param)

('init', ['he_normal', 'glorot_uniform'])
('dropout_rate', [0.2, 0.3, 0.5])
('adam_beta1', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
('adam_beta2', [0.101, 0.201, 0.301, 0.401, 0.501, 0.601, 0.701, 0.801, 0.901])


In [85]:
import itertools

# Extract the keys and values from the parameters dictionary
keys = parameters.keys()
values = parameters.values()

# Generate the Cartesian product of the parameter values
combinations = list(itertools.product(*values))

# Convert each combination into a dictionary
combinations_dicts = [dict(zip(keys, combo)) for combo in combinations]

# Example to print the first few combinations
for combo in combinations_dicts[:5]:
    print(combo)

print(len(combinations_dicts))

{'init': 'he_normal', 'dropout_rate': 0.2, 'adam_beta1': 0.1, 'adam_beta2': 0.101}
{'init': 'he_normal', 'dropout_rate': 0.2, 'adam_beta1': 0.1, 'adam_beta2': 0.201}
{'init': 'he_normal', 'dropout_rate': 0.2, 'adam_beta1': 0.1, 'adam_beta2': 0.301}
{'init': 'he_normal', 'dropout_rate': 0.2, 'adam_beta1': 0.1, 'adam_beta2': 0.401}
{'init': 'he_normal', 'dropout_rate': 0.2, 'adam_beta1': 0.1, 'adam_beta2': 0.501}
486


In [86]:
for combo in combinations_dicts:
    combo['lr'] = learning_rate
    model = unet(**combo)
    combo['history'] = model.fit(train_batches,
        validation_data=val_batches,
        epochs=50,
        verbose=2
    )

Epoch 1/50
20/20 - 108s - 5s/step - accuracy: 0.6984 - dice_metric: 0.0015 - loss: 0.9985 - val_accuracy: 0.5359 - val_dice_metric: 0.0026 - val_loss: 0.9974
Epoch 2/50
20/20 - 101s - 5s/step - accuracy: 0.8555 - dice_metric: 3.3477e-04 - loss: 0.9997 - val_accuracy: 0.8709 - val_dice_metric: 2.6473e-13 - val_loss: 1.0000
Epoch 3/50
20/20 - 100s - 5s/step - accuracy: 0.8685 - dice_metric: 1.5695e-06 - loss: 1.0000 - val_accuracy: 0.8709 - val_dice_metric: 3.3809e-13 - val_loss: 1.0000
Epoch 4/50


KeyboardInterrupt: 